####  В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [62]:
import numpy as np
from scipy import stats
from statsmodels.stats.weightstats import *

In [63]:
n = 50
p0 = float(34)/n

n1 = 34.
p1 = float(10)/n1
n2 = 16.
p2 = float(4)/n2

print('p1 = %f') % p1
print('p2 = %f') % p2

p1 = 0.294118
p2 = 0.250000


# Z-критерий для разности долей (независимые выборки)

$X_1$ ~ $Ber(p_1)$

$X_2$ ~ $Ber(p_2)$

$$H_0: p_1=p_2$$
$$H_1: p_1>p_2$$
$$p_{value} = 1 - F_{N(0,1)}(z)$$

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [64]:
P = (p1*n1 + p2*n2)/float(n1 + n2)
z = (p1 - p2)/np.sqrt(P*(1-P)*(1./n1 + 1./n2))
p_value = 1 - stats.norm.cdf(np.abs(z))

print p_value
print round(p_value, 4)

0.372930458725
0.3729


####  Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. 
Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

    логистическая регрессия по признакам X1,X2,X3;
    логистическая регрессия по признакам X4,X5,X6.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [65]:
import pandas as pd
from sklearn import cross_validation
from sklearn import linear_model

In [66]:
data_bank = pd.read_csv('banknotes.txt', sep="\t")
data_bank.describe()
data_bank.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [67]:
X = data_bank[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]
y = data_bank['real'].values
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=50, random_state=1)

In [68]:
print  len(X_train)
print  len(X_test)

150
50


In [69]:
X_train1 = X_train[['X1', 'X2', 'X3']].values
X_train2 = X_train[['X4', 'X5', 'X6']].values
X_test1 = X_test[['X1', 'X2', 'X3']].values
X_test2 = X_test[['X4', 'X5', 'X6']].values

log1 = linear_model.LogisticRegression(random_state=1)
log1.fit(X_train1, y_train)
pred1 = log1.predict(X_test1)
err1 = abs(pred1 - y_test)

log2 = linear_model.LogisticRegression(random_state=1)
log2.fit(X_train2, y_train)
pred2 = log2.predict(X_test2)
err2 = abs(pred2 - y_test)

err_prop1 = float(np.sum(err1)) / len(X_test)
err_prop2 = float(np.sum(err2)) / len(X_test)

print ('err1:', np.sum(err1))
print ('err2:', np.sum(err2))
print err_prop1
print err_prop2

('pred1:', array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0]))
('pred2:', array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0]))
('y:', array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0]))
('err1:', 10)
('err2:', 1)
0.2
0.02


$X_1$ ~ $Ber(p_1)$

$X_2$ ~ $Ber(p_2)$

$$H_0: p_1=p_2$$
$$H_1: p_1<>p_2$$
$$Z-статистика: Z({X_1, X_2}) =  \frac{f - g}{\sqrt{f + g - \frac{(f - g)^2}{n}}}$$


In [71]:
def proportions_linked_z(x1, x2):
    n = len(x1)
    f = np.sum([1 if (x == 1 and x2[i] == 0) else 0 for i, x in enumerate(x1)])
    g = np.sum([1 if (x == 0 and x2[i] == 1) else 0 for i, x in enumerate(x1)])
    z = float(f - g) / sqrt(f + g - float((f - g)**2) / n)
    return z

def proportions_linked_p_value(z):
    return 2*(1 - stats.norm.cdf(np.abs(z)))

z = proportions_linked_z(err1, err2)
print 'z = ', z
p = proportions_linked_p_value(z)
print 'p_value = ', p

('f:', 10)
('g:', 1)
('n:', 50)
z =  2.93860416802
p_value =  0.00329693845555


#### В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [76]:
def proportions_linked_int(x1, x2, alpha):
    n = len(x1)
    f = np.sum([1 if (x == 1 and x2[i] == 0) else 0 for i, x in enumerate(x1)])
    g = np.sum([1 if (x == 0 and x2[i] == 1) else 0 for i, x in enumerate(x1)])
    
    z_alpha = scipy.stats.norm.ppf(1 - alpha / 2.)
    left = float(f - g)/n - z_alpha*sqrt(float(f + g)/n**2 - float((f - g)**2)/n**3)
    right = float(f - g)/n + z_alpha*sqrt(float(f + g)/n**2 - float((f - g)**2)/n**3)
    
    return (left, right)

interv = proportions_linked_int(err1, err2, 0.05)
print ('95%% confidence interval: [%f, %f]' % interv)
print round(min(abs(np.array([interv[0], interv[1]]))), 4)

95% confidence interval: [0.059945, 0.300055]
0.0599


#### Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

$$H_0: \mu = \mu_0$$
$$H_1: \mu > \mu_0$$
$$Z(X) = \frac{\hat{X} - \mu_0}{\sigma / \sqrt{n}}$$

In [80]:
mu0 = 525.
sigma0 = 100.
n = 100
mu = 541.4

In [85]:
z = float(mu - mu0) / (sigma0/sqrt(n))
p = 1 - stats.norm.cdf(abs(z))
print 'p = %f' % p
print round(p, 4)

p = 0.050503
0.0505


#### Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [88]:
mu2 = 541.5
z = float(mu2 - mu0) / (sigma0/sqrt(n))
p = 1 - stats.norm.cdf(abs(z))
print 'p = %f' % p
print round(p, 4)

p = 0.049471
0.0495
